# ADVANCED OPENCV

###### AN XML FILE FOR THE REQUIRED CALSSIFIER MUST BE USED

FACE DETECTION

In [21]:
import cv2 as cv

img = cv.imread('Photos/group 2.jpg')

# MUST CONVERT TO GRAYSCALE FIRST
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# CREATE A HAAR CASCADE CLASSIFIER
haar_cascade = cv.CascadeClassifier('haar_face.xml')

# DETECT FACES
## haar_cascade.detectMultiScale(GRAY_IMG, SCALE_FACTOR, MIN_NEIGHBOURS)
## ==> RETURNS A LIST OF RECTANGLE COORDINATES: X_TLC, Y_TLC, WIDTH, HEIGHT
## SCALE_FACTOR ====> TWEAKS NOISE SENSITIVITY ===>
## MIN_NEIGHBOURS ==> TWEAKS NOISE SENSITIVITY ===> # OF NEIGHBOURS THE RECTANGLE SHOULD HAVE TO BE A FACE
faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 6)

print(f'# of found faces= {len(faces_rect)}')

# LETS DRAW A RECTANGLE AROUND FOUND FACES:
for (x, y, w, h) in faces_rect:
    
    cv.rectangle(img, (x, y), (x+w, y+h), (0,255,0), thickness= 2)

cv.imshow('detected faces', img)

cv.waitKey(0)

# of found faces= 6


-1

FACE RECOGNITION

In [6]:
# THIS FILE IS FOR TRAINING

import os
import cv2 as cv
import numpy as np


# TO CREATE A LIST OF FOLDER NAMES:
people = []
DIR = r'C:\Users\marco.beshay\OPENCV\opencv-course\Resources\Faces\train'

haar_cascade = cv.CascadeClassifier('haar_face.xml')

# r"STRING" ===> TO TREATED THE STRING AS A RAW STRING (\n ===> ARE NOT TREATED AS NEW LINE)
for i in os.listdir(DIR):
    people.append(i)

print(people)

# OR DECLARE MANUALLY
# people = ['Ben Afflek', 'Elton John', 'Jerry Seinfield', 'Madonna', 'Mindy  Kaling']

# WE NEED A SET OF FEATURES AND CORRESBONDING LABELS
features = []
labels = []

# LOOP THROUGH THE FOLDERS, IN EACH FOLDER LOOP THROUGH PHOTO FILES, DETECT FACES
def detect_and_label_faces():
    print('detecting faces/ features started')
    # LOOPING THROUGH FOLDERS:
    for person in people:
        # THIS ADDS THE FOLDER NAME (PERSON NAME) TO THE FOLDER PATH
        person_path = os.path.join(DIR, person)

        # THE LABEL IS SET TO THE INDEX OF THAT PERSON IN THE PEOPLE LIST TO REDUCE COMPUTING STRAIN
        label = people.index(person)

        # LOOP THROUGH EACH IMAGE IN THAT PERSONS FOLDER
        for img in os.listdir(person_path):
            # WE NEED EACH IMAGE'S PATH TO USE cv.imread(IMG_PATH)
            img_path = os.path.join(person_path, img)

            img_array = cv.imread(img_path)

            gray_img_array = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)

            faces_rect = haar_cascade.detectMultiScale(gray_img_array, scaleFactor = 1.1, minNeighbors = 4)

            # LOOP THROUGH FACES
            for (x, y, w, h) in faces_rect:
                # CROP THE FACE REGION OF INTEREST FROM THE PHOTO
                faces_roi = gray_img_array[y:y+h, x:x+w]

                # APPEND THE FACES TO THE FEATURES AND THE CORESPONDING LABEL
                features.append(faces_roi)
                labels.append(label)
    
    print('detecting faces/ features ended')


detect_and_label_faces()

# TO CHECK:
# print(f'length of features = {len(features)}')
# print(f'length of labels = {len(labels)}')

# INSTANTIATE THE FACE RECOGNISER
face_recogniser = cv.face.LBPHFaceRecognizer_create()

# LET'S CONVERT THE FEATURES AND LABELS LISTS TO NP ARRAYS
features = np.array(features, dtype='object')
labels = np.array(labels)

# TRAIN THE RECOGNISER ON THE LIST OF FEATURES AND LABLES
face_recogniser.train()

# YOU CAN SAVE THIS TRAINING IN A yml SOURCE FILE TO USE LATER
face_recogniser.save('face_trained.yml')

# YOU CAN SAVE THE FEATURES LIST AND LABELS LIST AS WELL
np.save('features.npy', features)
np.save('labels.npy', labels)

['Ben Afflek', 'Elton John', 'Jerry Seinfield', 'Madonna', 'Mindy Kaling']
detecting faces/ features started
detecting faces/ features ended


NameError: name 'face_recogniser' is not defined

In [ ]:
# THIS FILE IS FOR ACTUAL RECOGNITION

import os
import cv2 as cv
import numpy as np

haar_cascade = cv.CascadeClassifier('haar_face.xml')

people = ['Ben Afflek', 'Elton John', 'Jerry Seinfield', 'Madonna', 'Mindy  Kaling']
# USE allow_pickle = True WHEN LOADING OBJECT ARRAYS
features = np.load('features.npy', allow_pickle = True)
labels = np.load('labels.npy')

face_recogniser = cv.face.LBPHFaceRecognizer_create()

face_recogniser.read('face_trained.yml')

# NOW LET'S TEST THE MODEL WITH A NEW IMG
img = cv.imread(r'C:\Users\marco.beshay\OPENCV\opencv-course\Resources\Faces\val\ben_afflek\1.jpg')

gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

face_rect = haar_cascade.detectMultiScale(gray_img, scaleFactor = 1.1, minNeighbors = 4)

# LOOP THROUGH THE FACES FOUND:
for (x, y, w, h) in face_rect:
    # CROP THE FACE REGION OF INTEREST FROM THE PHOTO
    faces_roi = gray_img[y:y+h, x:x+w]

    label, confidence = face_recogniser.predict(faces_roi)

    print(f'label = {label} with confidence of {confidence}')

    # DISPLAY THE PERSON NAME AND DRAW A RECTANGLE ON THEIR FACE
    cv.putText(img, str(people[label]), (20, 20), cv.FONT_HERSHEY_COMPLEX, 1.0, (0, 255, 0), thickness = 2)
    cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), thickness = 2)

cv.imshow('detected face', img)

cv.waitKey(0)